# Gradient Descent for Linear Regression

We will implement and Linear Regression with Cost Function and Gradient Descent

### Tools

We will use:
* Numpy
* Matplotlib
* And some utils from COursera's Machine Learning Specialization Course

In [2]:
import math, copy
import numpy as np
import matplotlib.pyplot as plt

# Problem:

Let's take these data points, a house with 1000 sqft sold at 300,000 dollars and a house with 2000 sqft sold for 500,000 dollars.

| Size (1000 sqft) | Price (1000s of dollars) |
|----------|----------|
| 1 | 300 |
| 2 | 500 |

In [4]:
# Load our data
X_train = np.array([1.0,2.0]) # Features
y_train = np.array([300.0, 500.0]) # Target Values

In [7]:
# Compute Cost (Mean Squared Error Cost)

def compute_cost(x,y,w,b):
    m = x.shape[0]
    cost = 0
    for i in range(m):
        f_wb = w * x[i] + b
        cost = cost + (f_wb - y[i]) ** 2
    total_cost = 1 / (2 * m) * cost
    return total_cost

# My Understanding of Gradient Descent:  

Gradient Descent can be used to minimize the cost function of any algorithm. 

The formula for Linear Regression is:

$$
f_{(w,b)}(x^i) = wx^{(i)} + b
$$

The formula for cost function for Linear Regression is:

$$
J_{(w,b)} = \frac{1}{2m}\sum_{i=0}^{m-1} (f_{w,b}(x^i) - y^i)^2 \tag{1}
$$

Gradient Descent formula is:

Repeat Until Convergence: { 
$$
w = w - \alpha * \frac {\partial J(w,b)}{\partial w}
$$
$$
b = b - \alpha * \frac {\partial J(w,b)}{\partial b}
$$
}

where paramters w,b and updated simeltaneously

The gradient is defined as:

$$
\frac {\partial J(w,b)}{\partial w} = \frac{1}{m} \sum_{i=0}^{m=1}(f_{w,b}(x^{(i)}) - y^{(i)})x^{(i)}
$$

$$
\frac {\partial J(w,b)}{\partial b} = \frac{1}{m} \sum_{i=0}^{m=1}(f_{w,b}(x^{(i)}) - y^{(i)})
$$

Here we can calculate the partial derivatives for all the paramters before updating any paramters

# Implementing Gradient Descent

We will implement gradient descent algorithm for one feature only

We will use three functions:

`compute_gradient:` For computing this formula $$
\frac {\partial J(w,b)}{\partial w} = \frac{1}{m} \sum_{i=0}^{m=1}(f_{w,b}(x^{(i)}) - y^{(i)})x^{(i)}
$$

$$
\frac {\partial J(w,b)}{\partial b} = \frac{1}{m} \sum_{i=0}^{m=1}(f_{w,b}(x^{(i)}) - y^{(i)})
$$

`compute_cost`: For computing cost 
$$
J_{(w,b)} = \frac{1}{2m}\sum_{i=0}^{m-1} (f_{w,b}(x^i) - y^i)^2 \tag{1}
$$

`gradient_descent`: Utilizing both of the above functions
